# Hands-on: Hadronization - Brick Analysis

In this hands-on session we want to analyze the output of X-SCAPE framework simulations with a brick instead of a full hydrodynamical simulation for runtime reasons. We shoot a 100 GeV parton through a medium of different length and study the effects of different hadronization routines.

In [ ]:
# prerequisites: plots and packages
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import fastjet as fj
import sparkx.Jetscape as spxJ
import sparkx.Histogram as spxHist

# put some settings for the output
width = 0.05
plotMarkerSize = 8
labelfontsize = 15
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [6., 4.5]
mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['xtick.top'] = True
mpl.rcParams['xtick.labelsize'] = 13
mpl.rcParams['xtick.major.width'] = 1.0
mpl.rcParams['xtick.minor.width'] = 0.8
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['xtick.direction'] = "in"
mpl.rcParams['ytick.right'] = True
mpl.rcParams['ytick.labelsize'] = 13
mpl.rcParams['ytick.major.width'] = 1.0
mpl.rcParams['ytick.minor.width'] = 0.8
mpl.rcParams['ytick.minor.visible'] = True
mpl.rcParams['ytick.direction'] = "in"
mpl.rcParams['legend.fontsize'] = 13
mpl.rcParams['legend.numpoints'] = 1
mpl.rcParams['font.size'] = 13
mpl.rcParams['savefig.format'] = "pdf"

In [ ]:
# load all the data and perform the cuts we will use throughout the brick analysis
PATH_HADRONIZATION_DATA_HYBRID_2FM = '../../../X-SCAPE/build/hadronization_results/hadrons_brick_hybrid_2fm_final_state_hadrons.dat'
PATH_HADRONIZATION_DATA_HYBRID_4FM = '../../../X-SCAPE/build/hadronization_results/hadrons_brick_hybrid_4fm_final_state_hadrons.dat'
PATH_HADRONIZATION_DATA_COLORLESS_2FM = '../../../X-SCAPE/build/hadronization_results/hadrons_brick_colorless_2fm_final_state_hadrons.dat'
PATH_HADRONIZATION_DATA_HYBRID_BASELINE = '../../../X-SCAPE/build/hadronization_results/hadrons_vacuum_hybrid_baseline_final_state_hadrons.dat'
PATH_HADRONIZATION_DATA_COLORLESS_BASELINE = '../../../X-SCAPE/build/hadronization_results/hadrons_vacuum_colorless_baseline_final_state_hadrons.dat'

pseudorapidity_range = (-2.6,2.6)
deta = np.abs(pseudorapidity_range[1]-pseudorapidity_range[0])
pt_cut = (0.2,None)
charged_hadron_pt = np.array([0.2, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.5, 7.0, 9.0, 12.0, 15.0, 18.0, 21.0, 25.0, 29.0])
N_events = 1000

## Transverse Momentum Spectra



In [ ]:
def create_spectrum(hadrons, pt_bins, N_ev, deta):
    hist = spxHist.Histogram(pt_bins)
    for event in range(len(hadrons)):
        for hadron in range(len(hadrons[event])):
            hist.add_value(hadrons[event][hadron].pt_abs())

    hist_bin_widths = hist.bin_width()
    hist_bin_centers = hist.bin_centers()
    err_hist = hist.statistical_error() / (N_ev * 2. * np.pi * hist_bin_centers * hist_bin_widths * deta)
    hist.scale_histogram(1./(N_ev * 2. * np.pi * hist_bin_centers * hist_bin_widths * deta))
    data_hist = hist.histogram()
    return hist_bin_centers, hist_bin_widths, data_hist, err_hist

def compute_error_of_subtracted_spectrum(e1, e2):
    squared_e1 = e1 ** 2
    squared_e2 = e2 ** 2
    squared_e_sum = squared_e1 + squared_e2
    e = np.sqrt(squared_e_sum)
    return e

def ratio_error(v1,e1,v2,e2):
  #v1, e1: numerator value and error
  #v2, e2: denominator value and error  
  error1 = e1/v2
  error2 = (e2/v2)*(v1/v2)
  error = np.sqrt(error1*error1+error2*error2)
  return error

In [ ]:
# compute spectra for the hybrid hadronization 2 fm brick
simulation1 = spxJ.Jetscape(PATH_HADRONIZATION_DATA_HYBRID_2FM)
data_hybrid_2fm = simulation1.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_status([11,12,21,22]).particle_objects_list()

simulation1_reco = spxJ.Jetscape(PATH_HADRONIZATION_DATA_HYBRID_2FM)
data_hybrid_2fm_reco = simulation1_reco.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_status([11,12]).particle_objects_list()

simulation1_frag = spxJ.Jetscape(PATH_HADRONIZATION_DATA_HYBRID_2FM)
data_hybrid_2fm_frag = simulation1_frag.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_status([21,22]).particle_objects_list()

simulation1_neg = spxJ.Jetscape(PATH_HADRONIZATION_DATA_HYBRID_2FM)
data_hybrid_2fm_neg = simulation1_neg.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_status([-11,-12,-21,-22]).particle_objects_list()

simulation1_neg_reco = spxJ.Jetscape(PATH_HADRONIZATION_DATA_HYBRID_2FM)
data_hybrid_2fm_neg_reco = simulation1_neg_reco.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_status([-11,-12]).particle_objects_list()

simulation1_neg_frag = spxJ.Jetscape(PATH_HADRONIZATION_DATA_HYBRID_2FM)
data_hybrid_2fm_neg_frag = simulation1_neg_frag.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_status([-21,-22]).particle_objects_list()

pt_hybrid_2fm, dpt_hybrid_2fm, spec_hybrid_2fm, spec_err_hybrid_2fm = create_spectrum(data_hybrid_2fm,charged_hadron_pt,N_events,deta)
pt_hybrid_2fm_reco, dpt_hybrid_2fm_reco, spec_hybrid_2fm_reco, spec_err_hybrid_2fm_reco = create_spectrum(data_hybrid_2fm_reco,charged_hadron_pt,N_events,deta)
pt_hybrid_2fm_frag, dpt_hybrid_2fm_frag, spec_hybrid_2fm_frag, spec_err_hybrid_2fm_frag = create_spectrum(data_hybrid_2fm_frag,charged_hadron_pt,N_events,deta)

pt_hybrid_2fm_neg, dpt_hybrid_2fm_neg, spec_hybrid_2fm_neg, spec_err_hybrid_2fm_neg = create_spectrum(data_hybrid_2fm_neg,charged_hadron_pt,N_events,deta)
pt_hybrid_2fm_neg_reco, dpt_hybrid_2fm_neg_reco, spec_hybrid_2fm_neg_reco, spec_err_hybrid_2fm_neg_reco = create_spectrum(data_hybrid_2fm_neg_reco,charged_hadron_pt,N_events,deta)
pt_hybrid_2fm_neg_frag, dpt_hybrid_2fm_neg_frag, spec_hybrid_2fm_neg_frag, spec_err_hybrid_2fm_neg_frag = create_spectrum(data_hybrid_2fm_neg_frag,charged_hadron_pt,N_events,deta)

#subtract the spectrum of negative hadrons from the real ones
spec_final_hybrid_2fm = spec_hybrid_2fm - spec_hybrid_2fm_neg
spec_final_hybrid_2fm_err = compute_error_of_subtracted_spectrum(spec_err_hybrid_2fm,spec_err_hybrid_2fm_neg)

spec_final_hybrid_2fm_reco = spec_hybrid_2fm_reco - spec_hybrid_2fm_neg_reco
spec_final_hybrid_2fm_err_reco = compute_error_of_subtracted_spectrum(spec_err_hybrid_2fm_reco,spec_err_hybrid_2fm_neg_reco)

spec_final_hybrid_2fm_frag = spec_hybrid_2fm_frag - spec_hybrid_2fm_neg_frag
spec_final_hybrid_2fm_err_frag = compute_error_of_subtracted_spectrum(spec_err_hybrid_2fm_frag,spec_err_hybrid_2fm_neg_frag)

In [ ]:
# compute spectra for the hybrid hadronization 4 fm brick
simulation2 = spxJ.Jetscape(PATH_HADRONIZATION_DATA_HYBRID_4FM)
data_hybrid_4fm = simulation2.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_status([11,12,21,22]).particle_objects_list()

simulation2_reco = spxJ.Jetscape(PATH_HADRONIZATION_DATA_HYBRID_4FM)
data_hybrid_4fm_reco = simulation2_reco.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_status([11,12]).particle_objects_list()

simulation2_frag = spxJ.Jetscape(PATH_HADRONIZATION_DATA_HYBRID_4FM)
data_hybrid_4fm_frag = simulation2_frag.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_status([21,22]).particle_objects_list()

simulation2_neg = spxJ.Jetscape(PATH_HADRONIZATION_DATA_HYBRID_4FM)
data_hybrid_4fm_neg = simulation2_neg.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_status([-11,-12,-21,-22]).particle_objects_list()

simulation2_neg_reco = spxJ.Jetscape(PATH_HADRONIZATION_DATA_HYBRID_4FM)
data_hybrid_4fm_neg_reco = simulation2_neg_reco.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_status([-11,-12]).particle_objects_list()

simulation2_neg_frag = spxJ.Jetscape(PATH_HADRONIZATION_DATA_HYBRID_4FM)
data_hybrid_4fm_neg_frag = simulation2_neg_frag.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_status([-21,-22]).particle_objects_list()

pt_hybrid_4fm, dpt_hybrid_4fm, spec_hybrid_4fm, spec_err_hybrid_4fm = create_spectrum(data_hybrid_4fm,charged_hadron_pt,N_events,deta)
pt_hybrid_4fm_reco, dpt_hybrid_4fm_reco, spec_hybrid_4fm_reco, spec_err_hybrid_4fm_reco = create_spectrum(data_hybrid_4fm_reco,charged_hadron_pt,N_events,deta)
pt_hybrid_4fm_frag, dpt_hybrid_4fm_frag, spec_hybrid_4fm_frag, spec_err_hybrid_4fm_frag = create_spectrum(data_hybrid_4fm_frag,charged_hadron_pt,N_events,deta)

pt_hybrid_4fm_neg, dpt_hybrid_4fm_neg, spec_hybrid_4fm_neg, spec_err_hybrid_4fm_neg = create_spectrum(data_hybrid_4fm_neg,charged_hadron_pt,N_events,deta)
pt_hybrid_4fm_neg_reco, dpt_hybrid_4fm_neg_reco, spec_hybrid_4fm_neg_reco, spec_err_hybrid_4fm_neg_reco = create_spectrum(data_hybrid_4fm_neg_reco,charged_hadron_pt,N_events,deta)
pt_hybrid_4fm_neg_frag, dpt_hybrid_4fm_neg_frag, spec_hybrid_4fm_neg_frag, spec_err_hybrid_4fm_neg_frag = create_spectrum(data_hybrid_4fm_neg_frag,charged_hadron_pt,N_events,deta)

#subtract the spectrum of negative hadrons from the real ones
spec_final_hybrid_4fm = spec_hybrid_4fm - spec_hybrid_4fm_neg
spec_final_hybrid_4fm_err = compute_error_of_subtracted_spectrum(spec_err_hybrid_4fm,spec_err_hybrid_4fm_neg)

spec_final_hybrid_4fm_reco = spec_hybrid_4fm_reco - spec_hybrid_4fm_neg_reco
spec_final_hybrid_4fm_err_reco = compute_error_of_subtracted_spectrum(spec_err_hybrid_4fm_reco,spec_err_hybrid_4fm_neg_reco)

spec_final_hybrid_4fm_frag = spec_hybrid_4fm_frag - spec_hybrid_4fm_neg_frag
spec_final_hybrid_4fm_err_frag = compute_error_of_subtracted_spectrum(spec_err_hybrid_4fm_frag,spec_err_hybrid_4fm_neg_frag)

In [ ]:
# compute spectrum for the colorless hadronization 2 fm brick
simulation3 = spxJ.Jetscape(PATH_HADRONIZATION_DATA_COLORLESS_2FM)
data_colorless_2fm = simulation3.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_objects_list()

pt_colorless_2fm, dpt_colorless_2fm, spec_colorless_2fm, spec_err_colorless_2fm = create_spectrum(data_colorless_2fm,charged_hadron_pt,N_events,deta)

Separate spectra for the individual simulations, where we differentiate hadrons from recombination and string fragmentation:

In [ ]:
# plot the 2 fm brick hybrid hadronization results
fig, ax1 = plt.subplots()

ax1.errorbar(pt_hybrid_2fm_reco, spec_final_hybrid_2fm_reco, fmt='s', xerr=dpt_hybrid_2fm_reco/2., yerr=spec_final_hybrid_2fm_err_reco, color='b', label=r"recombination")
ax1.errorbar(pt_hybrid_2fm_frag, spec_final_hybrid_2fm_frag, fmt='o', xerr=dpt_hybrid_2fm_frag/2., yerr=spec_final_hybrid_2fm_err_frag, color='k', label=r"string frag.")
ax1.errorbar(pt_hybrid_2fm, spec_final_hybrid_2fm, fmt='x', xerr=dpt_hybrid_2fm/2., yerr=spec_final_hybrid_2fm_err, color='g', label=r"all hadrons")

ax1.set_yscale('log')
ax1.legend(loc=1)
annotation_string = r"""2 fm brick, 100 GeV jet
ch. hadrons
$|\eta|\leq 2.6$, $p_\mathrm{T}\geq 0.2$ GeV
"""
ax1.annotate(annotation_string,xy=(0.5, 0.45), xycoords='axes fraction')
ax1.set_xlabel(r"$p_\mathrm{T}$ [GeV]")
ax1.set_ylabel(r"$\frac{1}{N_\mathrm{ev}}\,\frac{\mathrm{d}^2 N}{2\pi p_{\mathrm{T}}\mathrm{d}p_{\mathrm{T}}\mathrm{d}\eta}$ [1/GeV$^2$]")
plt.savefig('./brick_2fm_hybrid_hadron_spectrum_charged.pdf', bbox_inches='tight')

In [ ]:
# plot the 4 fm brick hybrid hadronization results
fig, ax1 = plt.subplots()

ax1.errorbar(pt_hybrid_4fm_reco, spec_final_hybrid_4fm_reco, fmt='s', xerr=dpt_hybrid_4fm_reco/2., yerr=spec_final_hybrid_4fm_err_reco, color='b', label=r"recombination")
ax1.errorbar(pt_hybrid_4fm_frag, spec_final_hybrid_4fm_frag, fmt='o', xerr=dpt_hybrid_4fm_frag/2., yerr=spec_final_hybrid_4fm_err_frag, color='k', label=r"string frag.")
ax1.errorbar(pt_hybrid_4fm, spec_final_hybrid_4fm, fmt='x', xerr=dpt_hybrid_4fm/2., yerr=spec_final_hybrid_4fm_err, color='g', label=r"all hadrons")

ax1.set_yscale('log')
ax1.legend(loc=1)
annotation_string = r"""4 fm brick, 100 GeV jet
ch. hadrons
$|\eta|\leq 2.6$, $p_\mathrm{T}\geq 0.2$ GeV
"""
ax1.annotate(annotation_string,xy=(0.5, 0.45), xycoords='axes fraction')
ax1.set_xlabel(r"$p_\mathrm{T}$ [GeV]")
ax1.set_ylabel(r"$\frac{1}{N_\mathrm{ev}}\,\frac{\mathrm{d}^2 N}{2\pi p_{\mathrm{T}}\mathrm{d}p_{\mathrm{T}}\mathrm{d}\eta}$ [1/GeV$^2$]")
plt.savefig('./brick_4fm_hybrid_hadron_spectrum_charged.pdf', bbox_inches='tight')

In colorless hadronization there is only string fragmentation, so we will just plot *all hadrons* in this case.

In [ ]:
# plot the 2 fm brick colorless hadronization results
fig, ax1 = plt.subplots()

ax1.errorbar(pt_colorless_2fm, spec_colorless_2fm, fmt='s', xerr=dpt_colorless_2fm/2., yerr=spec_err_colorless_2fm, color='g', label=r"all hadrons")

ax1.set_yscale('log')
ax1.legend(loc=1)
annotation_string = r"""2 fm brick, 100 GeV jet
ch. hadrons
$|\eta|\leq 2.6$, $p_\mathrm{T}\geq 0.2$ GeV
"""
ax1.annotate(annotation_string,xy=(0.5, 0.45), xycoords='axes fraction')
ax1.set_xlabel(r"$p_\mathrm{T}$ [GeV]")
ax1.set_ylabel(r"$\frac{1}{N_\mathrm{ev}}\,\frac{\mathrm{d}^2 N}{2\pi p_{\mathrm{T}}\mathrm{d}p_{\mathrm{T}}\mathrm{d}\eta}$ [1/GeV$^2$]")
plt.savefig('./brick_2fm_colorless_hadron_spectrum_charged.pdf', bbox_inches='tight')

In the next step we will compare the fraction of recombination between the two bricks of length 2 and 4 fm by taking the ratio between the recombination hadrons and all hadrons in the corresponding spectra.

In [ ]:

fraction_reco_2fm = spec_final_hybrid_2fm_reco / spec_final_hybrid_2fm
fraction_reco_4fm = spec_final_hybrid_4fm_reco / spec_final_hybrid_4fm

fraction_reco_2fm_err = ratio_error(spec_final_hybrid_2fm_reco,spec_final_hybrid_2fm_err_reco,spec_final_hybrid_2fm,spec_final_hybrid_2fm_err)
fraction_reco_4fm_err = ratio_error(spec_final_hybrid_4fm_reco,spec_final_hybrid_4fm_err_reco,spec_final_hybrid_4fm,spec_final_hybrid_4fm_err)

fig, ax1 = plt.subplots()

ax1.errorbar(pt_hybrid_2fm, fraction_reco_2fm, fmt='s', xerr=dpt_hybrid_2fm/2., yerr=fraction_reco_2fm_err, color='g', label=r"$l_{\mathrm{brick}}=2$ fm")
ax1.errorbar(pt_hybrid_4fm, fraction_reco_4fm, fmt='o', xerr=dpt_hybrid_4fm/2., yerr=fraction_reco_4fm_err, color='b', label=r"$l_{\mathrm{brick}}=4$ fm")

ax1.legend(loc=1)
annotation_string = r"""brick, 100 GeV jet
ch. hadrons
$|\eta|\leq 2.6$, $p_\mathrm{T}\geq 0.2$ GeV
"""
ax1.annotate(annotation_string,xy=(0.5, 0.45), xycoords='axes fraction')
ax1.set_xlabel(r"$p_\mathrm{T}$ [GeV]")
ax1.set_ylabel(r"fraction of recombination hadrons")
plt.savefig('./brick_fraction_reco_hadrons.pdf', bbox_inches='tight')


Direct comparison of the 2 fm bricks in hybrid and colorless hadronization.

In [ ]:
fig, ax1 = plt.subplots()

ax1.errorbar(pt_hybrid_2fm, spec_final_hybrid_2fm, fmt='s', xerr=dpt_hybrid_2fm/2., yerr=spec_final_hybrid_2fm_err, color='g', label=r"hybrid hadr.")
ax1.errorbar(pt_colorless_2fm, spec_colorless_2fm, fmt='o', xerr=dpt_colorless_2fm/2., yerr=spec_err_colorless_2fm, color='b', label=r"colorless hadr.")

ax1.set_yscale('log')
ax1.legend(loc=1)
annotation_string = r"""2 fm brick, 100 GeV jet
ch. hadrons
$|\eta|\leq 2.6$, $p_\mathrm{T}\geq 0.2$ GeV
"""
ax1.annotate(annotation_string,xy=(0.5, 0.45), xycoords='axes fraction')
ax1.set_xlabel(r"$p_\mathrm{T}$ [GeV]")
ax1.set_ylabel(r"$\frac{1}{N_\mathrm{ev}}\,\frac{\mathrm{d}^2 N}{2\pi p_{\mathrm{T}}\mathrm{d}p_{\mathrm{T}}\mathrm{d}\eta}$ [1/GeV$^2$]")
plt.savefig('./brick_hybrid_colorless_comparison.pdf', bbox_inches='tight')

Compute the $p / \pi$ ratio in the simulation with the 2 fm brick for hybrid hadronization and colorless hadronization.

In [ ]:
# hybrid hadronization
data_hybrid_pions = spxJ.Jetscape(PATH_HADRONIZATION_DATA_HYBRID_2FM)
data_hybrid_2fm_pions = data_hybrid_pions.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_status([11,12,21,22]).particle_species([211,-211]).particle_objects_list()

data_hybrid_pions_neg = spxJ.Jetscape(PATH_HADRONIZATION_DATA_HYBRID_2FM)
data_hybrid_2fm_pions_neg = data_hybrid_pions_neg.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_status([-11,-12,-21,-22]).particle_species([211,-211]).particle_objects_list()

data_hybrid_protons = spxJ.Jetscape(PATH_HADRONIZATION_DATA_HYBRID_2FM)
data_hybrid_2fm_protons = data_hybrid_protons.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_status([11,12,21,22]).particle_species([2212,-2212]).particle_objects_list()

data_hybrid_protons_neg = spxJ.Jetscape(PATH_HADRONIZATION_DATA_HYBRID_2FM)
data_hybrid_2fm_protons_neg = data_hybrid_protons_neg.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_status([-11,-12,-21,-22]).particle_species([2212,-2212]).particle_objects_list()

pt_hybrid_2fm_pions, dpt_hybrid_2fm_pions, spec_hybrid_2fm_pions, spec_err_hybrid_2fm_pions = create_spectrum(data_hybrid_2fm_pions,charged_hadron_pt,N_events,deta)
pt_hybrid_2fm_pions_neg, dpt_hybrid_2fm_pions_neg, spec_hybrid_2fm_pions_neg, spec_err_hybrid_2fm_pions_neg = create_spectrum(data_hybrid_2fm_pions_neg,charged_hadron_pt,N_events,deta)

pt_hybrid_2fm_protons, dpt_hybrid_2fm_protons, spec_hybrid_2fm_protons, spec_err_hybrid_2fm_protons = create_spectrum(data_hybrid_2fm_protons,charged_hadron_pt,N_events,deta)
pt_hybrid_2fm_protons_neg, dpt_hybrid_2fm_protons_neg, spec_hybrid_2fm_protons_neg, spec_err_hybrid_2fm_protons_neg = create_spectrum(data_hybrid_2fm_protons_neg,charged_hadron_pt,N_events,deta)

#subtract the spectrum of negative hadrons from the real ones
spec_final_hybrid_2fm_pions = spec_hybrid_2fm_pions - spec_hybrid_2fm_pions_neg
spec_final_hybrid_2fm_pions_err = compute_error_of_subtracted_spectrum(spec_err_hybrid_2fm_pions,spec_err_hybrid_2fm_pions_neg)

spec_final_hybrid_2fm_protons = spec_hybrid_2fm_protons - spec_hybrid_2fm_protons_neg
spec_final_hybrid_2fm_protons_err = compute_error_of_subtracted_spectrum(spec_err_hybrid_2fm_protons,spec_err_hybrid_2fm_protons_neg)

p_to_pi_hybrid = spec_final_hybrid_2fm_protons / spec_final_hybrid_2fm_pions
p_to_pi_hybrid_err = ratio_error(spec_final_hybrid_2fm_protons,spec_final_hybrid_2fm_protons_err,spec_final_hybrid_2fm_pions,spec_final_hybrid_2fm_pions_err)

# colorless hadronization
data_colorless_pions = spxJ.Jetscape(PATH_HADRONIZATION_DATA_COLORLESS_2FM)
data_colorless_2fm_pions = data_colorless_pions.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_species([211,-211]).particle_objects_list()

data_colorless_protons = spxJ.Jetscape(PATH_HADRONIZATION_DATA_COLORLESS_2FM)
data_colorless_2fm_protons = data_colorless_protons.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_species([2212,-2212]).particle_objects_list()

pt_colorless_2fm_pions, dpt_colorless_2fm_pions, spec_colorless_2fm_pions, spec_err_colorless_2fm_pions = create_spectrum(data_colorless_2fm_pions,charged_hadron_pt,N_events,deta)

pt_colorless_2fm_protons, dpt_colorless_2fm_protons, spec_colorless_2fm_protons, spec_err_colorless_2fm_protons = create_spectrum(data_colorless_2fm_protons,charged_hadron_pt,N_events,deta)

p_to_pi_colorless = spec_colorless_2fm_protons / spec_colorless_2fm_pions
p_to_pi_colorless_err = ratio_error(spec_colorless_2fm_protons,spec_err_colorless_2fm_protons,spec_colorless_2fm_pions,spec_err_colorless_2fm_pions)

In [ ]:
fig, ax1 = plt.subplots()

ax1.errorbar(pt_hybrid_2fm_pions, p_to_pi_hybrid, fmt='s', xerr=dpt_hybrid_2fm_pions/2., yerr=p_to_pi_hybrid_err, color='g', label=r"hybrid hadr.")
ax1.errorbar(pt_colorless_2fm_pions, p_to_pi_colorless, fmt='o', xerr=dpt_colorless_2fm_pions/2., yerr=p_to_pi_colorless_err, color='b', label=r"colorless hadr.")

ax1.legend(loc=1)
annotation_string = r"""2 fm brick, 100 GeV jet
$|\eta|\leq 2.6$, $p_\mathrm{T}\geq 0.2$ GeV
"""
ax1.annotate(annotation_string,xy=(0.05, 0.8), xycoords='axes fraction')
ax1.set_xlabel(r"$p_\mathrm{T}$ [GeV]")
ax1.set_ylabel(r"$p/\pi$")
plt.savefig('./brick_p_pi_ratio.pdf', bbox_inches='tight')


Compute $R_{\mathrm{brick},\mathrm{vacuum}}$, which is defined as the ratio of the brick spectrum with the vacuum baseline.

In [ ]:
# load the vacuum baseline data and create spectra
simulation4 = spxJ.Jetscape(PATH_HADRONIZATION_DATA_HYBRID_BASELINE)
data_hybrid_baseline = simulation4.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_status([11,12,21,22]).particle_objects_list()

simulation4_neg = spxJ.Jetscape(PATH_HADRONIZATION_DATA_HYBRID_BASELINE)
data_hybrid_baseline_neg = simulation4_neg.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_status([-11,-12,-21,-22]).particle_objects_list()

pt_hybrid_2fm_baseline, dpt_hybrid_2fm_baseline, spec_hybrid_2fm_baseline, spec_err_hybrid_2fm_baseline = create_spectrum(data_hybrid_baseline,charged_hadron_pt,N_events,deta)
pt_hybrid_2fm_baseline_neg, dpt_hybrid_2fm_baseline_neg, spec_hybrid_2fm_baseline_neg, spec_err_hybrid_2fm_baseline_neg = create_spectrum(data_hybrid_baseline_neg,charged_hadron_pt,N_events,deta)

#subtract the spectrum of negative hadrons from the real ones
spec_final_hybrid_2fm_baseline = spec_hybrid_2fm_baseline - spec_hybrid_2fm_baseline_neg
spec_final_hybrid_2fm_baseline_err = compute_error_of_subtracted_spectrum(spec_err_hybrid_2fm_baseline,spec_err_hybrid_2fm_baseline_neg)


simulation5 = spxJ.Jetscape(PATH_HADRONIZATION_DATA_COLORLESS_BASELINE)
data_colorless_baseline = simulation5.remove_particle_species(22).charged_particles().pt_cut(pt_cut).pseudorapidity_cut(pseudorapidity_range).particle_objects_list()

pt_colorless_2fm_baseline, dpt_colorless_2fm_baseline, spec_colorless_2fm_baseline, spec_err_colorless_2fm_baseline = create_spectrum(data_colorless_baseline,charged_hadron_pt,N_events,deta)

R_bv_hybrid = spec_final_hybrid_2fm / spec_final_hybrid_2fm_baseline
R_bv_hybrid_err = ratio_error(spec_final_hybrid_2fm,spec_final_hybrid_2fm_err,spec_final_hybrid_2fm_baseline,spec_final_hybrid_2fm_baseline_err)

R_bv_colorless = spec_colorless_2fm / spec_colorless_2fm_baseline
R_bv_colorless_err = ratio_error(spec_colorless_2fm,spec_err_colorless_2fm,spec_colorless_2fm_baseline,spec_err_colorless_2fm_baseline)

In [ ]:
fig, ax1 = plt.subplots()

ax1.errorbar(pt_hybrid_2fm_baseline, R_bv_hybrid, fmt='s', xerr=dpt_hybrid_2fm_baseline/2., yerr=R_bv_hybrid_err, color='g', label=r"hybrid hadr.")
ax1.errorbar(pt_colorless_2fm_baseline, R_bv_colorless, fmt='o', xerr=dpt_colorless_2fm_baseline/2., yerr=R_bv_colorless_err, color='b', label=r"colorless hadr.")

# Add the horizontal dashed line at y = 1
plt.axhline(y=1, color='black', linestyle='--')

ax1.legend(loc=1)
annotation_string = r"""2 fm brick, 100 GeV jet
$|\eta|\leq 2.6$, $p_\mathrm{T}\geq 0.2$ GeV
"""
ax1.annotate(annotation_string,xy=(0.5, 0.45), xycoords='axes fraction')
ax1.set_xlabel(r"$p_\mathrm{T}$ [GeV]")
ax1.set_ylabel(r"$R_{\mathrm{brick},\mathrm{vacuum}}$")
plt.savefig('./brick_R_brick_vacuum_ratio.pdf', bbox_inches='tight')

## Homework 1

Run a jet finding algorithm on your favourite brick data set using the anti $k_{\mathrm{T}}$-algorithm and plot a jet spectrum. You can use most of the code from the $e^+e^-$ jupyter notebook. If you don't give a `jet_algorithm` parameter to the `JetAnalysis` class of the *sparkx* package, it will choose the anti $k_{\mathrm{T}}$-algorithm as a default.

## Homework 2

Play around with some parameters, e.g., the brick length or temperature, or change the energy of the jet. How do the spectra look like for a 20 GeV jet? For this exercise you have to modify the xml files or create new ones and then run some new simulations.